# Assessment 1:
## Get the square root of 2 and display to 100 decimal places

### Requirements:

Write a Python function called sqrt2 that calculates and prints to the screen the square root of 2 to 100 decimal places. Your code should not depend on any module from the standard library1 or otherwise. You should research the task first and include references and a description of your algorithm.

### Issue

There are two issues to be resolved as part of this assignment:

    1) How to display a floating point number accurately to 100 decimal places.
    2) How to calculate the square root of a number to 100 decimal places.
    
#### Display floating point number accurately to 100 decimal places

On computers, floating point numbers (numbers with a decimal place) are represented as fractions based on the base 2 (binary). In addition, the current standard for representing floating point numbers is a 64 bit system. Using the IEEE 754 standard (O'Sullivan), the most common representation of the floating point number is where:

    1st digit represents the sign of the number (positive or negative).
    2nd – 12th digit represents the number before the decimal point (the exponent).
    13th – 54th digits represent the numbers after the decimal point (the fraction).

The combination of these two issues represent a problem for trying to calculate any number to a degree of accuracy if they cannot be expressed as a binary number. This results in approximation of the value, a high degree, but not perfect accuracy.

For example, 0.5$_{10}$ (0.1$_{2}$) can easily be represented as a binary number, using the above standard as:

            0011 1111 0000 0000 0000 0000 0000 0000 (O'Sullivan)

However, the number 0.1$_{10}$ cannot be represented to the same degree of accuracy. Converting 0.1$_{10}$ into base 2 results in:

            0.0001100110011001100110011001100110011001100110011 (Regan, 2012)

When reconverted back into base 10, this results in a value of:

            0.1000000000000000055511151231257827021181583404541015625 (Regan, 2012)

Kahan (1997) notes that in a floating point number, the number of significant decimal places that can be achieved in 15 – 17 (p. 4). In the above base 10 representation, if the number were rounded to 17 decimal places, it becomes:

            0.10000000000000001 (Regan, 2012)

This implies that even though we can calculate a number to up to 50 decimal places, rounding a number to even 17 decimal places will produce a rounding error, and will not return the same number with which we started.

Dealing specifically with the square root of two, it is known that it is an irrational number (i.e. the pattern of the numbers does not repeat, unlike the pattern of 0011 in the value of 0.1($_{2}$) (Alfeld, 1996). Like above, even though the decimal output for the square root of 2 can be shown to approx. 53 decimal places, the rounding issue identified will mean that after about 17 significant digits, the number produced by any method involving floating point numbers will result in inaccuracies. This can be seen in the Newton-Raphson method below.


In [4]:
import datetime as dt                                                           # Only used for calculating speed of function

def NR_method(Number):

    root = Number / 2                                                           # Set the initial root value at half of the number to be square rooted
    Count = 1                                                                   # Count of number of iterations
    precision = 10 ** (-15)                                                     # Level of precision required

    while abs(Number - root * root) > 10 ** (-15):                              # While the Number less the detrived root is less than the precision

        root = (root + Number / root) / 2                                       # Find the new root through the Newton Raphson method
        Count += 1                                                              # Increment the count by 1

    return root, Count                                                          # Return the root and Count

Time_1 = dt.datetime.now()                                                      # Time at the start of the process

Number = 2                                                                      # Number whose square root is to be determined.

NR_SQRT, Iteration = NR_method(Number)                                          # Call the NR_method function, and assign the outputs to variables NR_SQRT and Iteration

Time_2 = dt.datetime.now()                                                      # Time at the end of the process

print()
print("Newton-Raphson Method")
print("-----------------------------------------------------------")
print("Square root 2 to 100 decimal places (Newton-Raphson method): \n%.100f" % NR_SQRT)
print()
print("Steps required to run this to get to 100 decimal places: %d" % Iteration)
print()
print("Time taken to run this method:", (Time_2-Time_1))


Newton-Raphson Method
-----------------------------------------------------------
Square root 2 to 100 decimal places (Newton-Raphson method): 
1.4142135623730949234300169337075203657150268554687500000000000000000000000000000000000000000000000000

Steps required to run this to get to 100 decimal places: 6

Time taken to run this method: 0:00:00


#### Issue with the Newton-Raphson method

The calculated Newton-Raphson method diverges from the "official" NASA version after 17 decimal places, where the method returns 0949 instead od 0950. This is due to issues within Python, where floats do not generally display more than 17 decimal places, even if they hold more.

Additionally, event when the variables hold more than 17 decimal places, they cannot hold more than 50 decimal places. This results in all the digits after the 50th decimal place being printed as 0. This can be seen in the above print out, when the float was specified at 100 decimal places. This issue can be resolved by using imported libraries, which can force more decimal places to be held. However that was not allowed by the rules of the assignment.

### Option 2: Subtraction method

This method is an old Japanese method described by <a href ="http://www.afjarvis.staff.shef.ac.uk/maths/jarvisspec02.pdf"> Frazer Jarvis</a>. The method involves an involves the use of 2 variables, "A" and "B". The initial value of the variable "A" is 5n (where n is the number for which the square root is being sought). In this case "A" would be equal to 10 (5n --> 5 x 2 = 10). The variable "B" can be any number, but the most accurate (through trial and error) is 5. 

The logic behind the method is:

    + If A is greater than or equal to B, change the value of A to A - B, and add 10 to B.
    + If B is greater than A, multiply A by 100, and place a 0 in the B variable, between the penultimate and final digit.

The first iterations of this method, for the square root of 2, would be:

    1) A = 10, B = 5  --> A > B  --> A = A - B = 10 - 5 = 5,  B = B + 10 = 5 + 10 = 15
    2) A = 5, B = 15  --> A < B  --> A = A x 100 = 5 x 100 = 500,  B = B[0] & "0" & B[1] = 1 & 0 & 5 = 105
    etc.....

Through this subtraction (and multiplication) method, the value of B becomes the square root of the value of n. However, it is still required to place the decimal place in the correct spot.

In [5]:
import datetime as dt

def sub_root(a, b, Count):                                      # Create a function to import values of a and b.
    while len(str(b)) <= 102:                                   # While the string length of b is less than 101 (1 integter, plus the 101 significant decimal place).

        if a >= b:                                              # If input "a" is greater than or equal input "b".
            a = a - b                                           # Set value of "a" to be "a" minus "b".
            b = b + 10                                          # Set value of "b" to be "b" + 10.

        elif a < b:                                             # If input "a" is less than input "b".
            a = a * 100                                         # Multiply "a" by 100.
            c = len(str(b))-1                                   # Set the value "c" to be the string length of "b" minus 1.
            d = str(b)                                          # Set the value "d" to be the string "b".
            e = int(d[:c] + "0" + d[c:])                        # Set the value of "e" to be the first "c" number of characters of the string "d".
                                                                # concatenated with a "0" and finished with the last digit of the string "d".
            b = e                                               # Set the value of "b" to be the same as "e".

        Count += 1                                              # Increment count by 1

        return sub_root(a, b, Count)                            # Return the value to the function to reprocess.

    return b, Count                                             # Return the values for "b" and Count when the above while loop is completed.

Time_1 = dt.datetime.now()                                      # Time at the start of the process

Number = 2                                                      # Number whose square root is to be determined.

SQRT, Count = sub_root(Number * 5, 5, 0)                        # Call the sub_root function, and assign the outputs to variables SQRT and Count
SQRT = str(SQRT)                                                # Turn the SQRT variable into a string from an integer
formatted_SQRT = f'{SQRT[0]}.{SQRT[1:101]}'                     # Set the value of "format_SQRT to be the value of "SQRT" with a decimal place after the 1st digit in the string.

Time_2 = dt.datetime.now()                                      # Time at the end of the process

print()
print("Subtraction method")
print("-----------------------------------------------------------")
print("Square root 2 to 100 decimal places: \n%s" % formatted_SQRT)
print()
print("Steps required to run this method to get to 100 decimal places: %d" % Count)
print()
print("Time taken to run this method:", (Time_2-Time_1))


Subtraction method
-----------------------------------------------------------
Square root 2 to 100 decimal places: 
1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727

Steps required to run this method to get to 100 decimal places: 583

Time taken to run this method: 0:00:00.001003


#### Notes on approach 2

The subraction method has an advantage over the Newton-Raphson method in that the value of B will always be greater than 0. This removes the issue, for Python, of the number of decimal places that can be calculated accurately, or displayed. In addition, by converting the number into a string, and placing a decimal place in the correct location, the method gives the impression of having been calculated by using floating point numbers, but has not been.

There are a few issues with this method:

1) The method requires the addition of a decimal place in the second position of the string. This is to convert it from 1.4E100 to 1.4..., as is the correct answer. For any number, whose square root is being sought, less than 100, there is no need to amend the code. For numbers greater than 100 there would need to be a change made in order to determine where the decimal place should go.

2) The method takes longer, in terms of iterations and time, to get to 100 decimal places than the Newton-Raphson method. A quick comparison between the two shows that it takes the Newton-Raphson method about 0.000000 seconds to run, and only requires 6 iterations of the code. The subratction method takes about 0.001 seconds to run, but requires 583 iterations of the code.

3) The function used is a recursive function. After a certain number of recursions, there is a is a risk of stack overflow due to the depth of the recursive function (tail-call). While this can be reduced or eliminated by setting the recursive depth of the stack to be higher than the defaul in Python.

### Check on the accuracy of the two methods

##### NASA
It is important to check on the accuracy of the two methods. The quickest methof is to check the value against the known value of the square root of 2. For this we can use the value of the square root of 2, as provided by <a href="https://apod.nasa.gov/htmltest/gifcity/sqrt2.1mil">NASA</a>. As can be seen below for 100 decimal places the Newton-Raphson method, with only 17 decimals displaying accurately, is off by 2.22E-16 (1.5E-13 percent). The subtraction method matches perfectly to 100 decimal places.

In [6]:
# NASA calculated square root of 2

NASA = 1.4142135623730950488016887242096980785696718753769480731766797379907324784621070388503875343276415727

Calc_SQRT = float(formatted_SQRT)

print()
print("NASA check")
print("-----------------------------------------------------------")
print()
print("Difference between NASA calcualtions and Newton-Raphson method is: ", (NR_SQRT - NASA))
print("Percentage difference between NASA calcualtions and Newton-Raphson method is: %.20f%%" % (((NASA - NR_SQRT) / NASA) * 100))

print()
print("Difference between NASA calcualtions and the subtraction method is: ", (Calc_SQRT - NASA))
print("Percentage difference between NASA calcualtions and the subtraction method is: %.20f%%" % (((NASA - Calc_SQRT) / NASA) * 100))


NASA check
-----------------------------------------------------------

Difference between NASA calcualtions and Newton-Raphson method is:  -2.220446049250313e-16
Percentage difference between NASA calcualtions and Newton-Raphson method is: 0.00000000000001570092%

Difference between NASA calcualtions and the subtraction method is:  0.0
Percentage difference between NASA calcualtions and the subtraction method is: 0.00000000000000000000%


## Final note on task 1

The issue of the (in)accuracy of floating point numbers, once they go passed the 16th/17th decimal place, can very clearly be seen by the value calculated in the Newton-Raphson method. Without the use of libraries to force the accuracy to the required level, other methods must be found. 

In this case, the calculation of the value of the square root of 2 using an ever increasing interger, which was then converted to a string resolved the issue. However, this proved to be more time consuming than the Newton-Raphson method, however it was more accurate.

An alternative method, that waas not fuly explored in Python (it was in Excel) was to set the initial value of the number to 2E200. This should have resulted in the same outcome as the subtraction method, with the need to convert to a string and introduce a decimal point. This would have taken 308 iterations (in Excel) to complete.

# Bibliography for task 1
https://apod.nasa.gov/htmltest/gifcity/sqrt2.1mil The Square Root of Two to 1 Million Digits Robert Nemiroff and Jerry Bonnell

Source: http://www.afjarvis.staff.shef.ac.uk/maths/jarvisspec02.pdf https://www.mathblog.dk/project-euler-80-digits-irrational-square-roots/

Square Roots by Subtraction FRAZER JARVIS pages 119 - 122 Mathematical Spectrum 2004/2005 Volume 37 Number 3

https://projecteuler.net/problem=80

Square root digital expansion Show HTML problem content Problem 80

It is well known that if the square root of a natural number is not an integer, then it is irrational. The decimal expansion of such square roots is infinite without any repeating pattern at all.

The square root of two is 1.41421356237309504880..., and the digital sum of the first one hundred decimal digits is 475.

For the first one hundred natural numbers, find the total of the digital sums of the first one hundred decimal digits for all the irrational square roots.

1.14 Decimals, Floats, and Floating Point Arithmetic. http://anh.cs.luc.edu/python/hands-on/3.1/handsonHtml/float.html Hands-on-Python Tutorial © Copyright 2019, Dr. Andrew N. Harrington. Last updated on Jan 05, 2020. Created using Sphinx 1.3.1+. October 3rd, 2020

Floating Point Arithmetic: Issues and Limitations The Python Tutorial © Copyright 2001-2020, Python Software Foundation. The Python Software Foundation is a non-profit corporation. Please donate.
Oct 08, 2020

Period of the Continued Fraction of √n Marius Beceanu February 5, 2003 https://web.archive.org/web/20151221205104/http://web.math.princeton.edu/mathlab/jr02fall/Periodicity/mariusjp.pdf

https://apod.nasa.gov/htmltest/gifcity/sqrt2.1mil

# Assessment 2:
## Calculate the Chi-squared test for independence for the given table

### Requirements
The Chi-squared test for independence is a statistical hypothesis test like a t-test. It is used to analyse whether two categorical variables are independent. The Wikipedia article gives the table below as an example, stating the Chi-squared value based on it is approximately 24.6. Use scipy.stats to verify this value and calculate the associated p value. You should include a short note with references justifying your analysis in a markdown cell.

||A|B|C|D|Total|
|:--|:--|:--|:--|:--|:--|
|White Collar|90|60|104|95|349|
|Blue Collar|30|50|51|20|151|
|No Collar|30|40|45|35|150|
|Total|150|150|200|150|650|

### What is Chi-Squared testing for independence


In [7]:
# Script to determine the Chi-squared test results for the given table

from scipy import stats as st
import numpy as np

# Create the lists containing the values for each of the collars
WC = [90, 60, 104, 95]                                                  # White Collar
BC = [30, 50, 51, 20]                                                   # Blue Colloar
NC = [30, 40, 45, 35]                                                   # No Collar

# Create the contingency table
collars = np.array([WC, BC, NC])                                        # Use the lists to create a 2D array

# Return the chi squared values, probability, degrees of freedom, and expected tables
chi_sqr_stat, p_val, D_of_F, expect_t = st.chi2_contingency(collars)

print("Critical value of Chi squared")
print("%.3f" %chi_sqr_stat)
print()
print("P-value")
print("%.6f" %p_val)
print()
print("Degrees of Freedom")
print(D_of_F)
print()
print("Expected Table")
print(expect_t)
print()

Critical value of Chi squared
24.571

P-value
0.000410

Degrees of Freedom
6

Expected Table
[[ 80.53846154  80.53846154 107.38461538  80.53846154]
 [ 34.84615385  34.84615385  46.46153846  34.84615385]
 [ 34.61538462  34.61538462  46.15384615  34.61538462]]



In [8]:
# Determine the Critical stat value at two levels of probability

probability_1 = 0.05                                                    # Set probability value 1 to 5%
probability_2 = 0.01                                                    # Set probability value 2 to 1%

critical_stat_1 = st.chi2.ppf(1-probability_1, D_of_F)                  # Calculate the test_stats for 6 degrees of freedom, and 95% confidence
critical_stat_2 = st.chi2.ppf(1-probability_2, D_of_F)                  # Calculate the test_stats for 6 degrees of freedom, and 99% confidence

print("At %.0f degrees of freedom and a probability of %d%%, the critical value is %.3f" % (D_of_F, (probability_1 * 100), critical_stat_1))
print("At %.0f degrees of freedom and a probability of %d%%, the critical value is %.3f" % (D_of_F, (probability_2 * 100), critical_stat_2))

At 6 degrees of freedom and a probability of 5%, the critical value is 12.592
At 6 degrees of freedom and a probability of 1%, the critical value is 16.812


### What does this mean?

1) <b><u>P-value</b></u>: The P-value calculated for the contingency table is 0.000410 (or 0.041%). This value is significantly below the probability vales of 0.05 (5%) and 0.01 (1%), it would suggest that the likelihood of getting such results randomly is extremely unlikely if the null hypothesis is correct. 
   
2) <b><u>Degrees of Freedom</b></u>: The calculated degrees of freedom is 6, which is the same as the degrees of freedom on the source page.

3) <b><u>Critical stat value</b></u>: The critical stat  values is calculated as 24.571, and is given as 24.6 on the Wikipedia page. These results are the same, if rounding to only 1 significant decimal place is allowed. However, as noted above the critical stat values for 6 degrees of freedom, and the p-values of 0.05 and 0.01 are 12.592, abd 16.812 respectively. Given that the calculated value is significantly higher than the expected values, it would suggest that the null hypothesis is not valid.

### Overall

As per the assignment, the calculated critical stats value was found to be the same as the given value (24.571 / 24.6, as per point 3 above). In addition the p-value that was requested is 0.000410 (as per point 1 above).

The overall result of the Chi-Squared testing suggests that the null hypothesis (*"that each person's neighborhood of residence is independent of the person's occupational classification"*) should be rejected, due to a p-value significantly lower than the probablity limits, and a critical stat value considerably higher than the Chi-Squared distribution tables would allow. As such, the alternative hypothesis *that each person's neighbourhood of residence is dependent on the person's occupational classification* must hold true.

References:
https://reneshbedre.github.io/blog/chisq.html

https://en.wikipedia.org/wiki/Chi-squared_test

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html

https://machinelearningmastery.com/chi-squared-test-for-machine-learning/

https://towardsdatascience.com/running-chi-square-tests-in-python-with-die-roll-data-b9903817c51b

https://www.geeksforgeeks.org/python-pearsons-chi-square-test/

https://www.pythonfordatascience.org/chi-square-test-of-independence-python/

https://pythonfordatascienceorg.wordpress.com/chi-square-python/

https://medium.com/@alok.ranjan4/chi-square-test-using-python-7f8f14731cb

https://www.spss-tutorials.com/chi-square-independence-test/

# Assessment 3:
## Standard deviation - population vs sample variation

### Requirements
The standard deviation of an array of numbers x is calculated using numpy as np.sqrt(np.sum((x - np.mean(x))**2)/len(x)). However, Microsoft Excel has two different versions of the standard deviation calculation, STDEV.P and STDEV.S. The STDEV.P function performs the above calculation but in the STDEV.S calculation the division is by len(x)-1 rather than len(x). 

Research these Excel functions, writing a note in a Markdown cell about the difference between them. Then use numpy to perform a simulation demonstrating that the STDEV.S calculation is a better estimate for the standard deviation of a population when performed on a sample.

### What is standard deviation?

### Population vs sample

#### Accuracy

In [9]:
import numpy as np 
from numpy import random
import pandas as pd 

# Calculate the population and standard deviation
Sample_size = [5, 10, 50, 100, 1000, 10000, 10000]                              # Number of variables in list

for i in range(len(Sample_size)):                                               # For evey sample size listed above
    variable_list = random.normal(0, 1, Sample_size[i])                         # Create a random list of 100000 values with a mean of 0, and a standard deviation of 1
    mean_of_variable_list = np.mean(variable_list)                              # Determine the mean of the list (should be zero, but will vary depending on the list generated)

    list_df = pd.DataFrame(variable_list, columns = ["Variables"])              # Create a datafram with the variable list as the only column
    list_df["Mean"] = mean_of_variable_list                                     # Create a new column populated with the mean of the variable
    list_df["Variance"] = ((list_df["Variables"] - list_df["Mean"])**2)         # Determine the variance for each row as square of the variable less the mean

    Pop_var = np.sqrt(sum(list_df["Variance"]) / len(variable_list))            # Determine the population standard deviation as the square root of the sum of the variances, divided by the sample size
    Sam_var = np.sqrt(sum(list_df["Variance"]) / (len(variable_list) - 1))      # Determine the sample standard deviation as the square root of the sum of the vairances, divided by the sample size - 1

    print("Number of variables in the list:", Sample_size[i])                   # Print the number of variables in the list
    print()
    print("The mean of the sample is %.4f" % (mean_of_variable_list))           # Print the mean of the sample
    print()

    print("Population standard deviation:")                                     # Values for the population standard deviation to 2, 4, 6, and 8 decimal places
    print("2 decimals\t4 decimals\t6 decimals\t8 decimals")                             
    print("%.2f\t\t%.4f\t\t%.6f\t%.8f" %(Pop_var, Pop_var, Pop_var, Pop_var))
    print()
    print("Sample standard deviation:")                                         # Values for the sample standard deviation to 2, 4, 6, and 8 decimal places
    print("2 decimals\t4 decimals\t6 decimals\t8 decimals")
    print("%.2f\t\t%.4f\t\t%.6f\t%.8f" %(Sam_var, Sam_var, Sam_var, Sam_var))
    print("___________________________________________________________")
    print()
    print()

Number of variables in the list: 5

The mean of the sample is 0.0247

Population standard deviation:
2 decimals	4 decimals	6 decimals	8 decimals
1.06		1.0646		1.064610	1.06460962

Sample standard deviation:
2 decimals	4 decimals	6 decimals	8 decimals
1.19		1.1903		1.190270	1.19026974
___________________________________________________________


Number of variables in the list: 10

The mean of the sample is -0.3349

Population standard deviation:
2 decimals	4 decimals	6 decimals	8 decimals
0.68		0.6754		0.675443	0.67544314

Sample standard deviation:
2 decimals	4 decimals	6 decimals	8 decimals
0.71		0.7120		0.711980	0.71197958
___________________________________________________________


Number of variables in the list: 50

The mean of the sample is -0.2399

Population standard deviation:
2 decimals	4 decimals	6 decimals	8 decimals
1.13		1.1315		1.131505	1.13150544

Sample standard deviation:
2 decimals	4 decimals	6 decimals	8 decimals
1.14		1.1430		1.142993	1.14299310
_________________

### Conclusion